In [20]:
from collections import Counter, OrderedDict
from itertools import product
import matplotlib.pyplot as plt
from random import choices

import numpy as np
import string
import sys
import re
import math

with open('Coco Script.txt', encoding='utf-8') as f:
    data = f.read()

data = data.lower()
data = data.translate(str.maketrans('', '', string.punctuation))
data = re.sub('[^a-z]+', ' ', data)
data = ' '.join(data.split(' '))

allchar = ' ' + string.ascii_lowercase

In [21]:
unigram = Counter(data)
unigram_prob = {ch: round((unigram[ch]) / (len(data)), 4) for ch in allchar}

uni_list = [unigram_prob[c] for c in allchar]

In [22]:
print((np.array(uni_list) > 0).all())
diff = 1 - sum(uni_list)
uni_list[0] += diff
np.savetxt("uni.txt", np.reshape(np.array(uni_list), (1, -1)), fmt='%.4f', delimiter=',')

True


In [25]:
def ngram(n):
    # all possible n-grams
    d = dict.fromkeys([''.join(i) for i in product(allchar, repeat=n)],0)
    # update counts
    d.update(Counter([''.join(j) for j in zip(*[data[i:] for i in range(n)])]))
    return d

bigram = ngram(2)
bigram_prob = {c: bigram[c] / unigram[c[0]] for c in bigram}
np.savetxt("big.txt", np.array(list(bigram_prob.values())).reshape((27, 27)), fmt='%.4f', delimiter=',')

In [26]:
bigram_prob = {c: round((bigram[c]+1) / (unigram[c[0]]+27), 4) for c in bigram} # p(b|a)
big_list = list(bigram_prob.values())
big_list[big_list==0]+=0.0001
print((np.array(big_list) > 0).all())
big_array = np.array(list(bigram_prob.values())).reshape((27, 27))
big_array[:,0] += 1-np.sum(big_array, axis=1)
big_array[0,0] += 0.0002
big_array[0,1] -= 0.0002
print((np.array(big_list) > 0).all())
np.savetxt("big.txt", big_array, fmt='%.4f', delimiter=',')

True
True


In [27]:
bigram_prob = {c: round((bigram[c]+1) / (unigram[c[0]]+27), 4) for c in bigram} # p(b|a)
t = 0
sum_value = 0.0
for key, value in bigram_prob.items():
    if value == 0:
        bigram_prob[key] += 0.0001
    t+=1
    sum_value += bigram_prob[key]
    if t%27 == 0:
        bigram_prob[key] += round(1-sum_value, 4)
        sum_value = 0.0

In [28]:
big_array = np.array(list(bigram_prob.values())).reshape((27, 27))
print((big_array > 0).all())
print(np.sum(big_array, axis=1))
np.savetxt("big.txt", big_array, fmt='%.4f', delimiter=',')

False
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1.]


In [29]:
bigram_prob[' z']+=0.0002
bigram_prob[' y']-=0.0002
bigram_prob['hz']+=0.0002
bigram_prob['hy']-=0.0002
bigram_prob['mz']+=0.0004
bigram_prob['my']-=0.0004
bigram_prob['sz']+=0.0002
bigram_prob['sy']-=0.0002

In [30]:
big_array = np.array(list(bigram_prob.values())).reshape((27, 27))
print((big_array > 0).all())
print(np.sum(big_array, axis=1))

True
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1.]


In [31]:
trigram = ngram(3)
trigram_prob = {c: round((trigram[c]+1) / (bigram[c[:2]]+27), 4) for c in trigram}
t = 0
sum_value = 0.0
for key, value in trigram_prob.items():
    if value == 0:
        trigram_prob[key] += 0.0001
    t+=1
    sum_value += trigram_prob[key]
    if t%27 == 0:
        trigram_prob[key] += round(1-sum_value, 4)
        sum_value = 0.0

In [32]:
tri_array = np.array(list(trigram_prob.values())).reshape((27*27, 27))
print((tri_array > 0).all())
print(np.sum(tri_array, axis=1))

False
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

In [33]:
trigram_prob[' tz'] += 0.0004
trigram_prob[' tw'] -= 0.0004
trigram_prob[' hz'] += 0.0005
trigram_prob[' hy'] -= 0.0005

In [34]:
tri_array = np.array(list(trigram_prob.values())).reshape((27*27, 27))
print((tri_array > 0).all())
print(np.sum(tri_array, axis=1))

True
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

## After smoothing!

In [35]:
def gen_bi(c):
    w = [bigram_prob[c + i] for i in allchar]
    return choices(allchar, weights=w)[0]
    

def gen_tri(ab):
    w_tri = [trigram_prob[ab + i] for i in allchar]
    return choices(allchar, weights=w_tri)[0]   


def gen_sen(c, num):
    res = c + gen_bi(c)
    for i in range(num - 2):
        if bigram[res[-2:]] == 0:
            t = gen_bi(res[-1])
        else:
            t = gen_tri(res[-2:])
        res += t
    return res

f = open("sen.txt", "w")
for i in string.ascii_lowercase:
    sentence = gen_sen(i, 1000)
    f.write(sentence+'\n')
f.close()

## Part 2

In [36]:
with open('script.txt', encoding='utf-8') as f:
    young = f.read() 

dict2 = Counter(young)
likeli = [dict2[c] / len(young) for c in allchar]
post_young = [round(0.3*likeli[i] / (0.3*likeli[i] + 0.7*uni_list[i]), 4) for i in range(27)]
post_hugh = [1 - post_young[i] for i in range(27)]

In [37]:
np.savetxt("q7.txt", np.array(likeli).reshape(1,27), fmt='%.4f', delimiter=',')

In [38]:
np.savetxt("q8.txt", np.array(post_young).reshape(1,27), fmt='%.4f', delimiter=',')

In [39]:
predicted = []
f = open('sen.txt','r')
for i in range(26):
    line = f.readline().strip('\n')
    counts = Counter(line)
    sorted(counts)
    sum_young = 0
    sum_hugh = 0
    for j in range(27):
        sum_young += math.log(post_young[j])*counts[j]
        sum_hugh += math.log(post_hugh[j])*counts[j]
    if sum_young > sum_hugh:
        predicted.append(1)
    else:
        predicted.append(0)
f.close()

In [42]:
np.savetxt("q9.txt", np.array(predicted).reshape(1,26), fmt='%d', delimiter=',')